In [12]:
import mlflow
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from process_and_ml.data_wrangler import load_azdias
import shap

In [2]:
train_df = pd.read_csv('../data/cleaned_data/train_cleaned.csv')
where_response = pd.read_csv('../data/train_test_files/Udacity_MAILOUT_052018_TRAIN.csv')

In [3]:
where_response = pd.read_csv('../data/train_test_files/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')


Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.


In [9]:
response = where_response['RESPONSE']

In [10]:
train_df_ok = train_df.iloc[:,1:]

In [14]:
from process_and_ml.pipeline import load_catboost_model, target_stats_by_feature


In [7]:
catboost = load_catboost_model('catboost_first.cbm')

In [11]:
d19_df = where_response['D19_SOZIALES']

In [21]:
from process_and_ml.pipeline import cat_features_fill_na

train_df_ok = cat_features_fill_na(train_df_ok, cat_features=['CAMEO_DEU_2015', 'OST_WEST_KZ'])


In [23]:
y_pred=catboost.predict_proba(train_df_ok)[:, 1]
predict =catboost.predict(train_df_ok)

In [26]:
print(f'Acuracia sem d19 = {accuracy_score(response, predict)}')
print(f'ROC normal = {roc_auc_score(response, y_pred)}')


Acuracia sem d19 = 0.687305060285834
ROC normal = 0.6065987499977848


In [27]:
df_19 = pd.DataFrame(d19_df)
df_19['pred'] = y_pred


In [112]:
df_19['pred_soc'] = np.where(df_19['D19_SOZIALES']==1, np.where(df_19['pred'] < 0.5, df_19['pred']*1.8, df_19['pred']), df_19['pred']*0.1)


In [113]:
print(f'ROC ncom 19 = {roc_auc_score(response, df_19.pred_soc.values)}')

ROC ncom 19 = 0.750858136089694


In [84]:
y_pred=catboost.predict_proba(train_df_ok)[:, 1]
predict =catboost.predict(train_df_ok)

ROC ncom 19 = 0.7467074252328914


In [8]:
df_19['wealth'] = train_df['WEALTH']

NameError: name 'df_19' is not defined

In [101]:
df_19['pred_soc'] = np.where(df_19['wealth']> 3, np.where(df_19['pred_soc'] < 0.8, df_19['pred_soc']*1.2, df_19['pred_soc']), df_19['pred_soc'])

In [73]:
print(f'ROC ncom 19 + wealth = {roc_auc_score(response, df_19.pred_soc.values)}')

ROC ncom 19 + wealth = 0.7408638996737661


In [122]:
for index, row in df_19.iterrows():
    if row['D19_SOZIALES']==1:
        if row['pred'] > 0.6:
            row['pred_soc'] = row['pred']*1.1
        elif 0.3 < row['pred'] < 0.7:
            row['pred_soc'] = row['pred']*1.8
        else:
            row['pred_soc'] = row['pred']*4
    elif row['D19_SOZIALES'] == 5 or row['D19_SOZIALES'] == 4 or row['D19_SOZIALES'] == 3 or row['D19_SOZIALES'] == 2:
        if row['pred'] > 0.6:
            row['pred_soc'] = row['pred']*0.6
        elif 0.3 < row['pred'] < 0.7:
            row['pred_soc'] = row['pred']*0.5
        else:
            row['pred_soc'] = row['pred']*0.2
    else:
            row['pred_soc'] = row['pred']


In [123]:
print(f'ROC ncom 19 weigths iterrows = {roc_auc_score(response, df_19.pred_soc.values)}')

ROC ncom 19 weigths iterrows = 0.7471900644848039


In [27]:

df_to_model_transformed = pd.concat([train_df_ok, response], axis=1)

In [38]:
target_stats_by_feature(df_to_model_transformed,
                        feature='CAMEO_DEU_2015',
                        target='RESPONSE',
                        fill_na_value=-1)

,CAMEO_DEU_2015,RESPONSE_mean,RESPONSE_count
2,1B,0.030457,197
5,1E,0.027397,365
1,1A,0.020260,691
19,5A,0.019272,467
22,5D,0.019060,787
21,5C,0.018957,422
16,4C,0.018953,2216
6,2A,0.018425,597
24,5F,0.016736,239
43,9D,0.016442,669


In [36]:
weights_column = {}
for each in df_to_model_transformed.columns:
    if each == 'RESPONSE':
        print(each)
    else:
        mean_df = target_stats_by_feature(df_to_model_transformed,
                                          feature=each,
                                          target='RESPONSE',
                                          fill_na_value=-1)
        mean_sum = mean_df.RESPONSE_mean.sum()
        weights_column[each] = mean_sum



sort_orders = sorted(weights_column.items(), key=lambda x: x[1], reverse=True)

RESPONSE


In [ ]:
## RODA COM O POPULATIONS TOTAL
